# Introduction
Building on the foundational CONFLUENCE workflow management and point-scale modeling principles established in Tutorial 01a, this notebook extends our analysis to focus on energy balance and evapotranspiration processes. While the previous SNOTEL tutorial emphasized snow dynamics and soil moisture in mountain environments, this example demonstrates CONFLUENCE's capabilities for simulating land-atmosphere interactions using eddy covariance flux tower observations from the global FLUXNET network.

## FLUXNET: Global Energy and Carbon Flux Observations
The FLUXNET network represents one of the most comprehensive global observational frameworks for studying land-atmosphere interactions, providing continuous measurements of energy, water, and carbon fluxes using the eddy covariance technique. These towers offer exceptional advantages for hydrological model evaluation through direct flux measurements that serve as validation targets for land surface energy balance models, high temporal resolution data that captures diurnal cycles and rapid environmental responses, multi-year records that enable assessment of seasonal dynamics and interannual variability, and ecosystem diversity spanning major biomes for process-based model evaluation across diverse vegetation types and climatic conditions.

## Scientific Importance of Energy Balance Modeling
Accurate representation of land-atmosphere energy exchanges forms the foundation of hydrological modeling through several critical processes. Evapotranspiration partitioning requires understanding the relative contributions of soil evaporation, plant transpiration, and canopy interception to total water loss. Energy balance processes directly influence soil moisture dynamics through evapotranspiration demand and soil-plant-atmosphere feedback mechanisms. Vegetation stress responses depend on accurate simulation of plant water stress and stomatal response to environmental conditions. These land-atmosphere interactions represent key feedback mechanisms in climate variability and change, making their accurate representation essential for robust hydrological predictions.

## Case Study: CA-NS7 Boreal Forest Site
This tutorial focuses on the CA-NS7 FLUXNET site located in the boreal forest of Saskatchewan, Canada (56.6358°N, 99.9483°W). This site presents distinct scientific challenges compared to the mountain snow environment of the previous tutorial. The site represents a mature boreal forest dominated by black spruce (Picea mariana) under a continental boreal climate regime with pronounced seasonal temperature variations. Located at 260 m elevation, the site features organic-rich soils with seasonal freezing and variable drainage conditions, typical of boreal landscapes.

## Learning Objectives
In this tutorial, you will extend CONFLUENCE applications to energy balance modeling and flux tower validation, understand ecosystem-specific modeling approaches for boreal forest conditions and vegetation parameterizations, evaluate energy balance processes by comparing simulated and observed evapotranspiration and sensible heat flux using established metrics, interpret land-atmosphere interactions through analysis of physical drivers underlying model-observation discrepancies in energy partitioning, and connect point-scale measurements to ecosystem scales by understanding how flux tower footprints relate to model grid cell assumptions.

This tutorial follows the established CONFLUENCE workflow while emphasizing energy balance processes through configuration adaptation for boreal forest conditions, integration of FLUXNET observations with meteorological forcing, SUMMA execution and flux evaluation comparing simulated and observed energy balance components, and detailed process analysis interpreting results within the context of boreal ecosystem dynamics. By completing this tutorial, you will develop understanding of energy balance modeling that complements the snow and soil moisture focus of the previous example, providing a comprehensive foundation for distributed hydrological modeling applications.

## Step 1: Rapid Workflow Setup for FLUXNET Energy Balance Modeling
Building on the CONFLUENCE fundamentals established in Tutorial 01a, we can now streamline the initial workflow setup. This step efficiently configures the system for energy balance validation at the CA-NS7 boreal forest FLUXNET site, leveraging the same reproducible framework.

In [ ]:
# Import the libraries we'll need in this notebook
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import xarray as xr
import numpy as np

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

# =============================================================================
# CONFIGURATION FOR CA-NS7 BOREAL FOREST SITE
# =============================================================================

# Set directory paths 
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/Users/darrieythorsson/compHydro/data/CONFLUENCE_data') 
#CONFLUENCE_DATA_DIR = Path('/path/to/your/CONFLUENCE_data') 

# Load template configuration and customize for FLUXNET site
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_template.yaml'

with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update for CA-NS7 boreal forest site
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'CA-NS7',
    'POUR_POINT_COORDS': '56.6358/-99.9483',  # CA-NS7 coordinates
    'DOWNLOAD_FLUXNET': 'true',
    'FLUXNET_STATION': 'CA-NS7',
    'EXPERIMENT_TIME_START': '2001-01-01 01:00',  # FLUXNET data availability
    'EXPERIMENT_TIME_END': '2005-12-31 23:00',
    'CALIBRATION_PERIOD': '2002-01-01, 2003-12-31',
    'EVALUATION_PERIOD': '2004-01-01, 2005-12-31',
    'SPINUP_PERIOD': '2001-01-01, 2001-12-31'
}

config_dict.update(config_updates)

# Save configuration
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_fluxnet_notebook.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"✅ Configuration saved: {temp_config_path}")

# =============================================================================
# SYSTEM INITIALIZATION AND PROJECT STRUCTURE
# =============================================================================

# Initialize CONFLUENCE with FLUXNET configuration
confluence = CONFLUENCE(temp_config_path)

## Step 2: Geospatial Domain Setup 
Having established the geospatial domain definition principles in Tutorial 01a, we can now efficiently configure the spatial framework for our boreal forest FLUXNET site. The same point-scale approach applies and the same CONFLUENCE spatial framework handles both environments seamlessly, demonstrating the transferability of the modeling approach across diverse ecosystems while capturing site-specific physical characteristics through the attribute acquisition process.

In [ ]:
# =============================================================================
# ATTRIBUTE ACQUISITION 
# =============================================================================

# confluence.managers['data'].acquire_attributes()

print("✅ Attribute acquisition complete")    

# =============================================================================
# DOMAIN DELINEATION AND DISCRETIZATION FOR POINT-SCALE FLUX TOWER
# =============================================================================

watershed_path = confluence.managers['domain'].define_domain()

# Domain discretization (single HRU for energy balance modeling)  
hru_path = confluence.managers['domain'].discretize_domain()

print(f"✅ Spatial domain configuration complete")

## Step 3: Data Pipeline
Leveraging the model-agnostic preprocessing concepts established in Tutorial 01a, we can now efficiently prepare the data pipeline for boreal forest energy balance modeling. The same standardized framework seamlessly handles the transition, demonstrating CONFLUENCE's versatility across diverse validation objectives.

In [ ]:
# Execute observational data processing
confluence.managers['data'].process_observed_data()
print("✅ Observed Data processing complete")

# Execute forcing acquisition 
# confluence.managers['data'].acquire_forcings()
print("✅ Forcing acquisition complete (simulated)")

# Execute model-agnostic preprocessing
confluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

# Execute model-specific preprocessing
confluence.managers['model'].preprocess_models()
print("✅ SUMMA configuration complete")

## Step 4: Model Execution 
Building on the detailed model instantiation concepts from Tutorial 01a, we can now efficiently execute the energy balance simulation. The same SUMMA process-based physics applies, but with emphasis on land-atmosphere energy exchange rather than snow accumulation and soil moisture dynamics.


The same workflow orchestration ensures reproducible and comparable simulations across both applications.

In [ ]:
# Execute the model
confluence.managers['model'].run_models()
print("✅ SUMMA simulation complete")

## Step 5: ET Process Validation
Building on the comprehensive evaluation framework established in Tutorial 01a, we now focus on energy flux validation using FLUXNET observations. The same scientific evaluation principles apply, but with emphasis on land-atmosphere energy exchange rather than snow/soil state variables.

In [ ]:
def process_fluxnet_data_inline(domain_name, data_dir):
    """Process raw FLUXNET data into standardized format for CONFLUENCE """
    
    # Set up paths
    data_dir = Path(data_dir)
    domain_dir = data_dir / f"domain_{domain_name}"
    raw_fluxnet_dir = domain_dir / "observations" / "fluxnet" / "raw_data"
    processed_dir = domain_dir / "observations" / "energy_fluxes" / "fluxnet" / "processed"
    
    # Create processed directory if it doesn't exist
    processed_dir.mkdir(parents=True, exist_ok=True)
        
    # Find FLUXNET files
    fluxnet_files = list(raw_fluxnet_dir.glob(f"FLX_{domain_name}_FLUXNET2015_FULLSET_*.csv"))
    
    if not fluxnet_files:
        print(f"❌ No FLUXNET files found in {raw_fluxnet_dir}")
        return False
    
    print(f"   Found {len(fluxnet_files)} FLUXNET files")
    
    # Process halfhourly data (most detailed for energy balance)
    hh_files = [f for f in fluxnet_files if "_HH_" in f.name]
    
    if not hh_files:
        print("❌ No halfhourly (_HH_) files found")
        return False
    
    file_path = hh_files[0]  # Use first halfhourly file
    
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        print(f"   Loaded {len(df)} rows, {len(df.columns)} columns")
        
        # Create timestamp from TIMESTAMP_START
        df['timestamp'] = pd.to_datetime(df['TIMESTAMP_START'].astype(str), format='%Y%m%d%H%M', errors='coerce')
        df = df.dropna(subset=['timestamp'])
        
        if len(df) == 0:
            print("❌ No valid timestamps found")
            return False
            
        # Key FLUXNET variables for energy balance
        key_variables = {
            'LE_F_MDS': 'Latent heat flux (gap-filled)',
            'H_F_MDS': 'Sensible heat flux (gap-filled)', 
            'RNET': 'Net radiation',
            'G_F_MDS': 'Ground heat flux (gap-filled)',
            'LE_F_MDS_QC': 'LE quality flag',
            'H_F_MDS_QC': 'H quality flag',
            'TA_F_MDS': 'Air temperature (gap-filled)',
            'PA_F': 'Atmospheric pressure',
            'WS_F': 'Wind speed',
            'RH': 'Relative humidity',
            'VPD_F_MDS': 'Vapor pressure deficit (gap-filled)',
            'SW_IN_F_MDS': 'Incoming shortwave radiation (gap-filled)',
            'P_F': 'Precipitation (gap-filled)',
        }
        
        # Select available variables
        available_vars = ['timestamp']
        for var in key_variables.keys():
            if var in df.columns:
                available_vars.append(var)
                
        # Create subset with available variables
        processed_df = df[available_vars].copy()
        
        # Replace FLUXNET missing value codes with NaN
        missing_value_codes = [-9999, -9999.0, -6999, -6999.0]
        for code in missing_value_codes:
            processed_df = processed_df.replace(code, np.nan)
        
        # Convert LE (W/m²) to ET (mm/day)
        if 'LE_F_MDS' in processed_df.columns:
            processed_df['ET_from_LE_mm_per_day'] = processed_df['LE_F_MDS'] * 0.0353
         
        # Calculate energy balance closure if components available
        energy_components = ['LE_F_MDS', 'H_F_MDS', 'G_F_MDS', 'RNET']
        if all(var in processed_df.columns for var in energy_components):
            processed_df['ENERGY_CLOSURE'] = (processed_df['LE_F_MDS'] + processed_df['H_F_MDS']) / (processed_df['RNET'] - processed_df['G_F_MDS'])
         
        # Save processed data
        output_file = processed_dir / f"{domain_name}_fluxnet_processed.csv"
        processed_df.to_csv(output_file, index=False)
        
        # Print data quality summary
        if 'LE_F_MDS' in processed_df.columns:
            valid_le = processed_df['LE_F_MDS'].notna().sum()
        
        if 'ET_from_LE_mm_per_day' in processed_df.columns:
            et_stats = processed_df['ET_from_LE_mm_per_day'].describe()
        
        return True
        
    except Exception as e:
        print(f"   ❌ Error processing {file_path.name}: {str(e)}")
        return False

# =============================================================================
# CHECK AND PROCESS FLUXNET DATA
# =============================================================================

print("\n🔧 Checking and Processing FLUXNET Data...")

# Check if processed FLUXNET data exists
fluxnet_processed_path = confluence.project_dir / "observations" / "energy_fluxes" / "fluxnet" / "processed" / f"{config_dict['DOMAIN_NAME']}_fluxnet_processed.csv"

if not fluxnet_processed_path.exists():
    print("⚠️  Processed FLUXNET data not found. Processing raw data...")
    
    # Process the data using our inline function
    success = process_fluxnet_data_inline(config_dict['DOMAIN_NAME'], str(CONFLUENCE_DATA_DIR))

# =============================================================================
# SIMULATION DATA LOADING 
# =============================================================================

# Load simulation data with proper filtering
sim_dir = confluence.project_dir / "simulations" / config_dict['EXPERIMENT_ID'] / "SUMMA"
sim_file = sim_dir / f"{config_dict['EXPERIMENT_ID']}_day.nc"
ds = xr.open_dataset(sim_file)

# Filter to experiment period
start_date = pd.to_datetime(config_dict['EXPERIMENT_TIME_START'])
end_date = pd.to_datetime(config_dict['EXPERIMENT_TIME_END'])

time_mask = (ds.time >= start_date) & (ds.time <= end_date)
evaluation_data = ds.isel(time=time_mask)
        
# Identify available energy balance variables
energy_variables = {
    'scalarLatHeatTotal': 'Latent heat flux (LE) - Evapotranspiration energy',
    'scalarSenHeatTotal': 'Sensible heat flux (H) - Convective energy transfer',
    'scalarNetRadiation': 'Net radiation (Rn) - Available energy',
    'scalarGroundHeatFlux': 'Ground heat flux (G) - Soil energy storage'
}

available_energy_vars = {var: desc for var, desc in energy_variables.items() 
                       if var in evaluation_data.data_vars}

# ET component variables for detailed analysis
et_components = {
    'scalarTotalET': 'Total evapotranspiration',
    'scalarCanopyTranspiration': 'Plant transpiration',
    'scalarCanopyEvaporation': 'Canopy interception evaporation', 
    'scalarGroundEvaporation': 'Soil surface evaporation',
    'scalarCanopySublimation': 'Canopy sublimation',
    'scalarSnowSublimation': 'Snow sublimation'
}

available_et_components = {var: desc for var, desc in et_components.items()
                         if var in evaluation_data.data_vars}

# =============================================================================
# FLUXNET OBSERVATION DATA LOADING
# =============================================================================

# Load processed FLUXNET data
if fluxnet_processed_path.exists():
    fluxnet_df = pd.read_csv(fluxnet_processed_path)
    fluxnet_df['timestamp'] = pd.to_datetime(fluxnet_df['timestamp'])
    fluxnet_df.set_index('timestamp', inplace=True)
    
    # Show key available variables (excluding QC flags)
    key_vars = [col for col in fluxnet_df.columns if not col.endswith('_QC') and col in 
                ['LE_F_MDS', 'H_F_MDS', 'RNET', 'G_F_MDS', 'ET_from_LE_mm_per_day', 'TA_F_MDS', 'VPD_F_MDS']]
    
    # Check data quality
    if 'ET_from_LE_mm_per_day' in fluxnet_df.columns:
        et_valid = fluxnet_df['ET_from_LE_mm_per_day'].notna().sum()
        
        et_stats = fluxnet_df['ET_from_LE_mm_per_day'].describe()
    
# =============================================================================
# ENERGY BALANCE EVALUATION: LATENT HEAT FLUX (ET)
# =============================================================================


# Check for ET data in simulation
et_var = None
conversion_factor = None

if 'scalarLatHeatTotal' in evaluation_data.data_vars:
    et_var = 'scalarLatHeatTotal'
    conversion_factor = 0.0353  # W/m² to mm/day
elif 'scalarTotalET' in evaluation_data.data_vars:
    et_var = 'scalarTotalET'
    conversion_factor = 86400  # kg m-2 s-1 to mm/day  

if et_var and fluxnet_df is not None and 'ET_from_LE_mm_per_day' in fluxnet_df.columns:
    
    # Extract simulated ET and convert units
    sim_et_xr = evaluation_data[et_var]
    
    # If multi-dimensional, take spatial mean first
    if len(sim_et_xr.dims) > 1:
        spatial_dims = [dim for dim in sim_et_xr.dims if dim != 'time']
        sim_et_xr = sim_et_xr.mean(dim=spatial_dims)
    
    # Convert to pandas Series
    sim_et_raw = sim_et_xr.to_pandas()
    
    # Handle negative values (SUMMA convention: negative = leaving system)
    median_val = sim_et_raw.median()
    if median_val < 0:
        sim_et_raw = -sim_et_raw
        print(f"   ⚡ Inverted sign for {et_var} (negative values indicate water leaving system)")
    
    sim_et_mm_day = sim_et_raw * conversion_factor
        
    # Find common period and align data
    common_start = max(fluxnet_df.index.min(), sim_et_mm_day.index.min())
    common_end = min(fluxnet_df.index.max(), sim_et_mm_day.index.max())
    
    # Resample to daily and filter to common period
    obs_daily = fluxnet_df['ET_from_LE_mm_per_day'].resample('D').mean().loc[common_start:common_end]
    sim_daily = sim_et_mm_day.resample('D').mean().loc[common_start:common_end]
    
    # Remove NaN values for metrics calculation
    valid_mask = ~(obs_daily.isna() | sim_daily.isna())
    obs_valid = obs_daily[valid_mask]
    sim_valid = sim_daily[valid_mask]
    
    if len(obs_valid) > 10:  # Need minimum data for meaningful analysis
        
        # Calculate performance metrics
        print(f"\n📊 Evapotranspiration Performance Metrics:")
        
        rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
        bias = (sim_valid - obs_valid).mean()
        mae = np.abs(obs_valid - sim_valid).mean()
        
        # Handle correlation calculation
        try:
            corr = obs_valid.corr(sim_valid)
            if pd.isna(corr):
                corr = 0.0
        except:
            corr = 0.0
        
        # Nash-Sutcliffe Efficiency
        if obs_valid.var() > 0:
            nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()
        else:
            nse = np.nan
        
        print(f"   RMSE: {rmse:.2f} mm/day")
        print(f"   Bias: {bias:+.2f} mm/day")
        print(f"   MAE: {mae:.2f} mm/day") 
        print(f"   Correlation: {corr:.3f}")
        print(f"   Nash-Sutcliffe Efficiency: {nse:.3f}")
        
        # Seasonal analysis
        print(f"\n🗓️ Seasonal ET Performance:")
        seasonal_data = pd.DataFrame({
            'obs': obs_valid,
            'sim': sim_valid,
            'month': obs_valid.index.month
        })
        
        seasonal_stats = seasonal_data.groupby('month').apply(
            lambda x: pd.Series({
                'obs_mean': x['obs'].mean(),
                'sim_mean': x['sim'].mean(),
                'bias': x['sim'].mean() - x['obs'].mean(),
                'corr': x['obs'].corr(x['sim']) if len(x) > 3 else np.nan
            })
        )
        
        seasons = [(12, 'Winter'), (3, 'Spring'), (6, 'Summer'), (9, 'Fall')]
        for month, label in seasons:
            if month in seasonal_stats.index:
                stats = seasonal_stats.loc[month]
                print(f"   {label:6s}: Obs={stats['obs_mean']:.2f}, Sim={stats['sim_mean']:.2f} mm/day, "
                      f"Bias={stats['bias']:+.2f}, r={stats['corr']:.3f}")
        
        # Create comprehensive ET visualization
        print(f"\n📈 Creating ET comparison visualization...")
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Time series comparison
        ax1 = axes[0, 0]
        obs_plot = obs_daily.dropna()
        sim_plot = sim_daily.dropna()
        
        ax1.plot(obs_plot.index, obs_plot.values, 'o-', label='FLUXNET ET', 
                 color='blue', alpha=0.7, markersize=1, linewidth=1)
        ax1.plot(sim_plot.index, sim_plot.values, '-', label='SUMMA ET', 
                 color='red', linewidth=2)
        ax1.set_title('Evapotranspiration Time Series', fontweight='bold')
        ax1.set_ylabel('ET (mm/day)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Scatter plot
        ax2 = axes[0, 1]
        ax2.scatter(obs_valid, sim_valid, alpha=0.6, c='green', s=20)
        max_val = max(obs_valid.max(), sim_valid.max())
        min_val = min(obs_valid.min(), sim_valid.min())
        ax2.plot([min_val, max_val], [min_val, max_val], 'k--', label='1:1 line')
        ax2.set_xlabel('FLUXNET ET (mm/day)')
        ax2.set_ylabel('SUMMA ET (mm/day)')
        ax2.set_title('Observed vs. Simulated ET', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Add metrics text
        metrics_text = f'r = {corr:.3f}\\nRMSE = {rmse:.2f}\\nBias = {bias:+.2f}'
        ax2.text(0.05, 0.95, metrics_text, transform=ax2.transAxes,
                 bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
        
        # Monthly climatology
        ax3 = axes[1, 0]
        monthly_obs = obs_valid.groupby(obs_valid.index.month).mean()
        monthly_sim = sim_valid.groupby(sim_valid.index.month).mean()
        months = range(1, 13)
        month_names = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
        
        # Ensure we have data for plotting
        full_monthly_obs = pd.Series(index=months, dtype=float)
        full_monthly_sim = pd.Series(index=months, dtype=float)
        
        for month in months:
            if month in monthly_obs.index:
                full_monthly_obs[month] = monthly_obs[month]
            if month in monthly_sim.index:
                full_monthly_sim[month] = monthly_sim[month]
        
        ax3.plot(months, full_monthly_obs, 'o-', label='FLUXNET', color='blue', linewidth=2)
        ax3.plot(months, full_monthly_sim, 'o-', label='SUMMA', color='red', linewidth=2)
        ax3.set_xticks(months)
        ax3.set_xticklabels(month_names)
        ax3.set_ylabel('ET (mm/day)')
        ax3.set_title('Monthly ET Climatology', fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # Residuals
        ax4 = axes[1, 1]
        residuals = sim_valid - obs_valid
        ax4.scatter(obs_valid.index, residuals, alpha=0.6, c='purple', s=15)
        ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        if residuals.std() > 0:
            ax4.axhline(y=residuals.std(), color='red', linestyle='--', alpha=0.5, label='+1σ')
            ax4.axhline(y=-residuals.std(), color='red', linestyle='--', alpha=0.5, label='-1σ')
            ax4.legend()
        ax4.set_ylabel('Residuals (mm/day)')
        ax4.set_title('Model Residuals', fontweight='bold')
        ax4.grid(True, alpha=0.3)
        
        plt.suptitle(f'Evapotranspiration Evaluation - {config_dict["DOMAIN_NAME"]} Boreal Forest', 
                     fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
# =============================================================================
# ET COMPONENT ANALYSIS
# =============================================================================

# Extract and convert ET components
et_comp_data = {}
for comp_var, description in available_et_components.items():
    comp_xr = evaluation_data[comp_var]
    
    # If multi-dimensional, take spatial mean first
    if len(comp_xr.dims) > 1:
        spatial_dims = [dim for dim in comp_xr.dims if dim != 'time']
        comp_xr = comp_xr.mean(dim=spatial_dims)
    
    # Convert to pandas Series
    comp_ts = comp_xr.to_pandas()
    
    # Handle negative values
    median_val = comp_ts.median()
    if median_val < 0:
        comp_ts = -comp_ts
    
    # Convert from kg m-2 s-1 to mm/day
    comp_ts_mm_day = comp_ts * 86400
    et_comp_data[comp_var] = comp_ts_mm_day
    
    print(f"   🌱 {comp_var}: {comp_ts_mm_day.mean():.3f} ± {comp_ts_mm_day.std():.3f} mm/day")

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Component time series (monthly means)
ax1 = axes[0]
colors = plt.cm.Set2.colors

for i, (comp_var, comp_data) in enumerate(et_comp_data.items()):
    if comp_var != 'scalarTotalET':  # Skip total for component plot
        monthly_comp = comp_data.resample('ME').mean()
        ax1.plot(monthly_comp.index, monthly_comp.values, 
                label=comp_var.replace('scalar', '').replace('Canopy', 'Can.').replace('Ground', 'Grd.'), 
                color=colors[i % len(colors)], linewidth=2, marker='o', markersize=3)

ax1.set_title('SUMMA ET Components - Monthly Means (Boreal Forest)', fontweight='bold')
ax1.set_ylabel('ET Component (mm/day)')
ax1.legend(loc='upper right', fontsize=9)
ax1.grid(True, alpha=0.3)

# Component seasonal climatology
ax2 = axes[1]
months = range(1, 13)
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for i, (comp_var, comp_data) in enumerate(et_comp_data.items()):
    if comp_var != 'scalarTotalET':
        monthly_mean = comp_data.groupby(comp_data.index.month).mean()
        
        # Ensure all months are represented
        full_monthly = pd.Series(index=months, dtype=float)
        for month in months:
            if month in monthly_mean.index:
                full_monthly[month] = monthly_mean[month]
            else:
                full_monthly[month] = 0.0
        
        ax2.plot(months, full_monthly.values, 'o-', linewidth=2,
                label=comp_var.replace('scalar', '').replace('Canopy', 'Can.').replace('Ground', 'Grd.'), 
                color=colors[i % len(colors)])

ax2.set_title('ET Component Seasonal Climatology (Boreal Forest)', fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('ET Component (mm/day)')
ax2.set_xticks(months)
ax2.set_xticklabels(month_names)
ax2.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate component contributions
print(f"\n📊 Annual ET Component Contributions:")
annual_totals = {}
for comp_var, comp_data in et_comp_data.items():
    if comp_var != 'scalarTotalET':
        annual_total = comp_data.resample('YE').sum().mean()  # Average annual total
        annual_totals[comp_var] = annual_total



total_annual = sum(annual_totals.values())
for comp_var, annual_total in annual_totals.items():
    percentage = (annual_total / total_annual) * 100 if total_annual > 0 else 0
    print(f"   🌿 {comp_var.replace('scalar', '')}: {annual_total:.1f} mm/yr ({percentage:.1f}%)")

## Tutorial Summary
### Point-Scale Energy Balance Validation
This tutorial successfully demonstrated CONFLUENCE's versatility by extending the established workflow framework from snow and soil moisture validation to comprehensive energy balance modeling using FLUXNET observations. Through the CA-NS7 boreal forest case study, we illustrated how the same standardized preprocessing pipeline and model execution framework seamlessly adapts across diverse ecosystems and validation objectives while maintaining scientific rigor and reproducibility.

### Key Methodological Advances
The tutorial showcased process-specific model configuration by adapting CONFLUENCE for boreal forest energy balance processes while maintaining the same underlying workflow structure. Multi-flux validation was achieved through comprehensive evaluation of evapotranspiration against direct eddy covariance measurements, demonstrating quantitative performance assessment across temporal scales from sub-daily to seasonal dynamics. Process component analysis revealed the relative contributions of different evapotranspiration pathways including transpiration, canopy evaporation, and soil evaporation, providing insights into boreal forest water cycling mechanisms.

### Scientific Process Understanding
The evaluation demonstrated CONFLUENCE's capability to simulate land-atmosphere energy exchange processes with explicit representation of stomatal conductance, canopy interception, and soil-plant-atmosphere feedback mechanisms. Seasonal energy dynamics were successfully captured, showing the model's ability to represent pronounced boreal forest phenological cycles and temperature-driven evapotranspiration patterns. Energy balance closure analysis provided direct validation of fundamental thermodynamic principles in the model physics.

### Framework Scalability Demonstration
This tutorial reinforced CONFLUENCE's transferable methodology by applying identical workflow principles across contrasting environments from mountain snow zones to boreal forests. The model-agnostic preprocessing approach proved equally effective for both snow/soil validation and energy flux evaluation, confirming the framework's broad applicability. Comparative process validation established the foundation for multi-site energy balance studies and ecosystem-specific model benchmarking that will be essential for larger-scale applications in subsequent tutorials.

### Next Focus: Watershed Simulations and Basin-Scale Processes
**Ready to explore Basin Scale simulations?** → **[Tutorial 02a: Basin Scale - Lumped Watershed](./02a_basin_lumped.ipynb)**